In [33]:
!pip install -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.6 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [pyLDAvis]5/6 [pyLDAvis]s]


In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re, string
from gensim import corpora
from gensim.models.ldamodel import LdaModel


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aeshagandhi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
import nltk
# Download required datasets
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aeshagandhi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/aeshagandhi/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aeshagandhi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aeshagandhi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/aeshagandhi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [21]:
data = pd.read_csv('data/cleaned_data.csv')
data.head()


,Unnamed: 0,statement,status,cleaned_statement
0,0,oh my gosh,Anxiety,oh my gosh
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety,trouble sleeping confused mind restless heart ...
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety,all wrong back off dear forward doubt stay in ...
3,3,I've shifted my focus to something else but I'...,Anxiety,i have shifted my focus to something else but ...
4,4,"I'm restless and restless, it's been a month n...",Anxiety,i am restless and restless it is been a month ...


### Step 1: Preprocess Text ###

In [22]:
# Basic cleaning
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)              # remove links
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words and w.isalpha()]
    return tokens

data['tokens'] = data['cleaned_statement'].astype(str).apply(clean_text)


In [23]:
data.head()

,Unnamed: 0,statement,status,cleaned_statement,tokens
0,0,oh my gosh,Anxiety,oh my gosh,"[oh, gosh]"
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety,trouble sleeping confused mind restless heart ...,"[trouble, sleeping, confused, mind, restless, ..."
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety,all wrong back off dear forward doubt stay in ...,"[wrong, back, dear, forward, doubt, stay, rest..."
3,3,I've shifted my focus to something else but I'...,Anxiety,i have shifted my focus to something else but ...,"[shifted, focus, something, else, still, worried]"
4,4,"I'm restless and restless, it's been a month n...",Anxiety,i am restless and restless it is been a month ...,"[restless, restless, month, boy, mean]"


### Step 2: Create Dictionary and Corpus for LDA ###

In [27]:
from gensim import corpora

#dictionary maps words to integer IDs.
dictionary = corpora.Dictionary(data['tokens'])
# corpus is a list of “bag-of-words” frequency vectors for each document.
corpus = [dictionary.doc2bow(text) for text in data['tokens']]


In [28]:
dictionary

### Step 3: Train LDA Model ###

In [29]:
from gensim.models.ldamodel import LdaModel

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=7,      # you can tune this
    passes=10,
    random_state=42
)


### Step 4: View the Topics ###

In [30]:
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")


Topic 0: 0.016*"take" + 0.013*"trying" + 0.012*"back" + 0.012*"experience" + 0.012*"started" + 0.011*"attack" + 0.010*"got" + 0.008*"went" + 0.008*"school" + 0.008*"would"
Topic 1: 0.024*"people" + 0.020*"want" + 0.020*"know" + 0.019*"life" + 0.016*"friend" + 0.015*"would" + 0.014*"one" + 0.013*"like" + 0.012*"even" + 0.011*"never"
Topic 2: 0.053*"feel" + 0.046*"like" + 0.019*"get" + 0.019*"feeling" + 0.014*"know" + 0.014*"day" + 0.013*"want" + 0.012*"even" + 0.012*"work" + 0.011*"time"
Topic 3: 0.018*"year" + 0.016*"time" + 0.013*"week" + 0.013*"really" + 0.011*"day" + 0.010*"get" + 0.009*"also" + 0.008*"last" + 0.008*"think" + 0.008*"going"
Topic 4: 0.044*"job" + 0.024*"wish" + 0.020*"deal" + 0.019*"ill" + 0.019*"money" + 0.019*"live" + 0.014*"state" + 0.013*"energy" + 0.012*"super" + 0.010*"eat"
Topic 5: 0.043*"fucking" + 0.041*"recently" + 0.038*"put" + 0.026*"either" + 0.024*"kill" + 0.020*"physical" + 0.018*"group" + 0.018*"shit" + 0.017*"type" + 0.015*"noticed"
Topic 6: 0.077*"a

### Step 5: Get Topic Distribution for each Document ###

In [31]:
topics_per_doc = [lda_model.get_document_topics(doc) for doc in corpus]

# Convert to a dense DataFrame
import numpy as np
topic_vectors = np.zeros((len(topics_per_doc), lda_model.num_topics))
for i, doc_topics in enumerate(topics_per_doc):
    for topic_num, prob in doc_topics:
        topic_vectors[i, topic_num] = prob

lda_features = pd.DataFrame(topic_vectors, columns=[f"Topic_{i}" for i in range(lda_model.num_topics)])
lda_features.head()


,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6
0,0.047655,0.047655,0.047655,0.047663,0.380485,0.381231,0.047655
1,0.017885,0.017870,0.892686,0.017906,0.017869,0.017867,0.017918
2,0.014329,0.326335,0.267466,0.348962,0.014317,0.014294,0.014294
3,0.020502,0.467824,0.200213,0.250132,0.020442,0.020442,0.020443
4,0.340306,0.024005,0.540274,0.023870,0.023821,0.023859,0.023866


In [35]:
data_with_topics = pd.concat([data.reset_index(drop=True), lda_features], axis=1)
data_with_topics.head()


,Unnamed: 0,statement,status,cleaned_statement,tokens,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6
0,0,oh my gosh,Anxiety,oh my gosh,"[oh, gosh]",0.047655,0.047655,0.047655,0.047663,0.380485,0.381231,0.047655
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety,trouble sleeping confused mind restless heart ...,"[trouble, sleeping, confused, mind, restless, ...",0.017885,0.017870,0.892686,0.017906,0.017869,0.017867,0.017918
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety,all wrong back off dear forward doubt stay in ...,"[wrong, back, dear, forward, doubt, stay, rest...",0.014329,0.326335,0.267466,0.348962,0.014317,0.014294,0.014294
3,3,I've shifted my focus to something else but I'...,Anxiety,i have shifted my focus to something else but ...,"[shifted, focus, something, else, still, worried]",0.020502,0.467824,0.200213,0.250132,0.020442,0.020442,0.020443
4,4,"I'm restless and restless, it's been a month n...",Anxiety,i am restless and restless it is been a month ...,"[restless, restless, month, boy, mean]",0.340306,0.024005,0.540274,0.023870,0.023821,0.023859,0.023866


### Visualize Topics ###

In [34]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.215175 -0.063303       1        1  29.713210
3      0.193707  0.008002       2        1  22.653481
2      0.206551 -0.017204       3        1  21.726258
0      0.092855  0.014854       4        1  13.234456
6     -0.169824  0.368316       5        1   4.857028
4     -0.205831 -0.209329       6        1   4.650702
5     -0.332633 -0.101336       7        1   3.164865, topic_info=         Term          Freq         Total Category  logprob  loglift
33       feel  35284.000000  35284.000000  Default  30.0000  30.0000
124   anxiety  14298.000000  14298.000000  Default  29.0000  29.0000
28       like  40746.000000  40746.000000  Default  28.0000  28.0000
230    people  19213.000000  19213.000000  Default  27.0000  27.0000
673       job   7469.000000   7469.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
790      shit   1462.174347   3082.185551   Topic7  -4.0414   2.7073
4497   common    503.888459    673.119322   Topic7  -5.1067   3.1635
176       god    542.330533   1177.853544   Topic7  -5.0332   2.6775
179    please    743.873068   3208.763877   Topic7  -4.7172   1.9913
1009   trying    579.162641   4948.828790   Topic7  -4.9675   1.3077

[450 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
4294       1  0.999239     act
4739       7  0.998738  acting
10310      3  0.025076    adhd
10310      4  0.003134    adhd
10310      5  0.970118    adhd
...      ...       ...     ...
376        3  0.042099    year
376        4  0.102319    year
376        5  0.003202    year
376        6  0.006199    year
10261      5  0.997598  zoloft

[759 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 1, 7, 5, 6])